<a href="https://colab.research.google.com/github/Rocking-Priya/704-fall-projects-2025/blob/main/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DX 704 Week 8 Project
This homework will modify a simulator controlling a small vehicle to implement tabular q-learning.
You will first test your code with random and greedy-epsilon policies, then tweak your own training method for a more optimal policy.

The full project description and a template notebook are available on GitHub: [Project 8 Materials](https://github.com/bu-cds-dx704/dx704-project-08).


## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Rover Simulator

The following Python class implements a simulation of a simple vehicle with integer x,y coordinates facing in one of 8 possible directions.


In [1]:
# DO NOT CHANGE

import random

class RoverSimulator(object):
    DIRECTIONS = ((0, 1), (1, 1), (1, 0), (1, -1), (0, -1), (-1, -1), (-1, 0), (-1, 1))

    def __init__(self, resolution):
        self.resolution = resolution
        self.terminal_state = self.construct_state(resolution // 2, resolution // 2, 0)

        self.initial_states = []
        for initial_x in (0, resolution // 2, resolution - 1):
            for initial_y in (0, resolution // 2, resolution - 1):
                for initial_direction in range(8):
                    initial_state = self.construct_state(initial_x, initial_y, initial_direction)
                    if initial_state != self.terminal_state:
                        self.initial_states.append(initial_state)

    def construct_state(self, x, y, direction):
        assert 0 <= x < self.resolution
        assert 0 <= y < self.resolution
        assert 0 <= direction < 8

        state = (y * self.resolution + x) * 8 + direction
        assert self.decode_state(state) == (x, y, direction)
        return state

    def decode_state(self, state):
        direction = state % 8
        x = (state // 8) % self.resolution
        y = state // (8 * self.resolution)

        return (x, y, direction)

    def get_actions(self, state):
        return [-1, 0, 1]

    def get_next_reward_state(self, curr_state, curr_action):
        if curr_state == self.terminal_state:
            # no rewards or changes from terminal state
            return (0, curr_state)

        (curr_x, curr_y, curr_direction) = self.decode_state(curr_state)
        (curr_dx, curr_dy) = self.DIRECTIONS[curr_direction]

        assert self.construct_state(curr_x, curr_y, curr_direction) == curr_state

        assert curr_action in (-1, 0, 1)

        next_x = min(max(0, curr_x + curr_dx), self.resolution - 1)
        next_y = min(max(0, curr_y + curr_dy), self.resolution - 1)
        next_direction = (curr_direction + curr_action) % 8

        next_state = self.construct_state(next_x, next_y, next_direction)
        next_reward = 1 if next_state == self.terminal_state else 0

        return (next_reward, next_state)

    def rollout_policy(self, policy_func, max_steps=1000):
        curr_state = self.sample_initial_state()
        for i in range(max_steps):
            curr_action = policy_func(curr_state, self.get_actions(curr_state))
            (next_reward, next_state) = self.get_next_reward_state(curr_state, curr_action)
            yield (curr_state, curr_action, next_reward, next_state)
            curr_state = next_state

    def sample_initial_state(self):
        return random.choice(self.initial_states)

In [13]:
simulator = RoverSimulator(16)
initial_sample = simulator.sample_initial_state()
print("INITIAL SAMPLE", initial_sample)

INITIAL SAMPLE 65


## Part 1: Implement a Random Policy

Random policies are often used to test simulators and start initial exploration.
Implement a random policy for these simulators.

In [14]:
# YOUR CHANGES HERE

def random_policy(state, actions):
    return random.choice(actions)


Use the code below to test your random policy.
Then modify it to save the results in "log-random.tsv" with the columns curr_state, curr_action, next_reward and next_state.

In [15]:
# YOUR CHANGES HERE

for (curr_state, curr_action, next_reward, next_state) in simulator.rollout_policy(random_policy, max_steps=32):
    print("CURR STATE", curr_state, "ACTION", curr_action, "NEXT REWARD", next_reward, "NEXT STATE", next_state)

CURR STATE 123 ACTION 1 NEXT REWARD 0 NEXT STATE 124
CURR STATE 124 ACTION -1 NEXT REWARD 0 NEXT STATE 123
CURR STATE 123 ACTION 0 NEXT REWARD 0 NEXT STATE 123
CURR STATE 123 ACTION 1 NEXT REWARD 0 NEXT STATE 124
CURR STATE 124 ACTION 1 NEXT REWARD 0 NEXT STATE 125
CURR STATE 125 ACTION -1 NEXT REWARD 0 NEXT STATE 116
CURR STATE 116 ACTION -1 NEXT REWARD 0 NEXT STATE 115
CURR STATE 115 ACTION -1 NEXT REWARD 0 NEXT STATE 122
CURR STATE 122 ACTION 1 NEXT REWARD 0 NEXT STATE 123
CURR STATE 123 ACTION 0 NEXT REWARD 0 NEXT STATE 123
CURR STATE 123 ACTION 0 NEXT REWARD 0 NEXT STATE 123
CURR STATE 123 ACTION -1 NEXT REWARD 0 NEXT STATE 122
CURR STATE 122 ACTION 1 NEXT REWARD 0 NEXT STATE 123
CURR STATE 123 ACTION -1 NEXT REWARD 0 NEXT STATE 122
CURR STATE 122 ACTION 1 NEXT REWARD 0 NEXT STATE 123
CURR STATE 123 ACTION 1 NEXT REWARD 0 NEXT STATE 124
CURR STATE 124 ACTION 0 NEXT REWARD 0 NEXT STATE 124
CURR STATE 124 ACTION 1 NEXT REWARD 0 NEXT STATE 125
CURR STATE 125 ACTION -1 NEXT REWARD 0 N

In [20]:
with open("log-random.tsv", "w") as f:
    f.write("curr_state\tcurr_action\tnext_reward\tnext_state\n")
    for (curr_state, curr_action, next_reward, next_state) in simulator.rollout_policy(random_policy, max_steps=32):
        f.write(f"{curr_state}\t{curr_action}\t{next_reward}\t{next_state}\n")


In [21]:
import google.colab.files

google.colab.files.download("log-random.tsv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Submit "log-random.tsv" in Gradescope.

## Part 2: Implement Q-Learning with Random Policy

The code below runs 32 random rollouts of 1024 steps using your random policy.
Modify the rollout code to implement Q-Learning.
Just implement one learning update for each sampled state-action in the simulation.
Use $\alpha=1$ and $\gamma=0.9$ since the simulator is deterministic and there is a sink where the rewards stop.




In [39]:
num_episodes = 32
max_steps = 1024
Q = {}                # dictionary: key=(state,action) -> float value
alpha = 1.0
gamma = 0.9

with open("q-random.tsv", "w") as f:
    # Write the header **once**
    f.write("curr_state\tcurr_action\tnext_reward\tnext_state\told_value\tnew_value\n")

    for episode in range(num_episodes):
        for (curr_state, curr_action, next_reward, next_state) in simulator.rollout_policy(random_policy, max_steps=max_steps):

            # old Q-value
            old_value = Q.get((curr_state, curr_action), 0.0)

            # best next Q-value
            next_actions = simulator.get_actions(next_state)
            best_next = max(Q.get((next_state, a), 0.0) for a in next_actions)

            # Q-learning update (alpha=1 → full replacement)
            new_value = next_reward + gamma * best_next
            Q[(curr_state, curr_action)] = new_value

            # write the data row
            f.write(f"{curr_state}\t{curr_action}\t{next_reward}\t{next_state}\t{old_value}\t{new_value}\n")


Save each step in the simulator in a file "q-random.tsv" with columns curr_state, curr_action, next_reward, next_state, old_value, new_value.

In [52]:
import pandas as pd
df = pd.read_csv("q-random.tsv", sep="\t")

print("Data rows:", len(df))  # should be 32768


Data rows: 32768


In [41]:
google.colab.files.download("q-random.tsv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Submit "q-random.tsv" in Gradescope.

## Part 3: Implement Epsilon-Greedy Policy

Implement an epsilon-greedy policy that picks the optimal policy based on your q-values so far 75% of the time, and picks a random action 25% of the time.
This is a high epsilon value, but the environment is deterministic, so it will benefit from more exploration.

In [48]:
# YOUR CHANGES HERE
Q = {}               # empty Q-table: key = (state, action) -> value
alpha = 1.0
gamma = 0.9
epsilon = 0.25
def epsilon_greedy_policy(state, actions):
    # With probability epsilon, pick a random action
    if random.random() < epsilon:
        return random.choice(actions)
    # Otherwise pick the best action according to current Q-values
    q_values = [Q.get((state, a), 0.0) for a in actions]
    max_q = max(q_values)
    best_actions = [a for a, qv in zip(actions, q_values) if qv == max_q]
    return random.choice(best_actions)

Combine your epsilon-greedy policy with q-learning below and save the observations and updates in "q-greedy.tsv" with columns curr_state, curr_action, next_reward, next_state, old_value, new_value.

Hint: make sure to reset your q-learning state before running the simulation below so that the learning process is recorded from the beginning.

In [49]:
num_episodes = 32
max_steps = 1024

with open("q-greedy.tsv", "w") as f:
    # write header
    f.write("curr_state\tcurr_action\tnext_reward\tnext_state\told_value\tnew_value\n")

    for episode in range(num_episodes):
        for (curr_state, curr_action, next_reward, next_state) in simulator.rollout_policy(
                epsilon_greedy_policy, max_steps=max_steps):

            old_value = Q.get((curr_state, curr_action), 0.0)
            next_actions = simulator.get_actions(next_state)
            best_next = max(Q.get((next_state, a), 0.0) for a in next_actions)
            new_value = next_reward + gamma * best_next
            Q[(curr_state, curr_action)] = new_value

            # log step
            f.write(f"{curr_state}\t{curr_action}\t{next_reward}\t{next_state}\t{old_value}\t{new_value}\n")




Submit "q-greedy.tsv" in Gradescope.

In [50]:
import pandas as pd
df = pd.read_csv("q-greedy.tsv", sep="\t")
print("Data rows:", len(df))  # should be 32768


Data rows: 32768


In [51]:
google.colab.files.download("q-greedy.tsv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Part 4: Extract Policy from Q-Values

Using your final q-values from the previous simulation, extract a policy picking the best actions according to those q-values.
Save the policy in a file "policy-greedy.tsv" with columns state and action.

In [29]:
# YOUR CHANGES HERE



with open("policy-greedy.tsv", "w") as f:
    f.write("state\taction\n")   # header

    # Use all states observed in Q (or simulator.initial_states)
    all_states = set(s for (s, a) in Q.keys())

    for state in all_states:
        actions = simulator.get_actions(state)
        # pick action(s) with max Q
        q_values = [Q.get((state, a), 0.0) for a in actions]
        max_q = max(q_values)
        best_actions = [a for a, qv in zip(actions, q_values) if qv == max_q]
        best_action = random.choice(best_actions)   # tie-break randomly
        f.write(f"{state}\t{best_action}\n")


In [30]:
google.colab.files.download("policy-greedy.tsv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Submit "policy-greedy.tsv" in Gradescope.

## Part 5: Implement Large Policy

Train a more optimal policy using q-learning.
Save the policy in a file "policy-optimal.tsv" with columns state and action.

Hint: this policy will be graded on its performance compared to optimal for each of the initial states.
**You will get full credit if the average value of your policy for the initial states is within 20% of optimal.**
Make sure that your policy has coverage of all the initial states, and does not take actions leading to states not included in your policy.
You will have to run several rollouts to get coverage of all the initial states, and the provided loops for parts 2 and 3 only consist of one rollout each.

Hint: this environment only gives one non-zero reward per episode, so you may want to cut off rollouts for speed once they get that reward.
But make sure you update the q-values first!

In [31]:
# YOUR CHANGES HERE
import random

# --- Reset Q-table ---
Q = {}
alpha = 1.0
gamma = 0.9
epsilon = 0.25   # can still use epsilon-greedy for exploration

# Epsilon-greedy policy for exploration
def epsilon_greedy_policy(state, actions):
    if random.random() < epsilon:
        return random.choice(actions)
    q_values = [Q.get((state, a), 0.0) for a in actions]
    max_q = max(q_values)
    best_actions = [a for a, qv in zip(actions, q_values) if qv == max_q]
    return random.choice(best_actions)

# --- Run many rollouts to cover all initial states ---
with open("policy-optimal.tsv", "w") as f:
    f.write("state\taction\n")  # header

    # Repeat enough episodes to cover all initial states
    for episode in range(200):   # increase number of episodes if needed
        for initial_state in simulator.initial_states:
            curr_state = initial_state
            for step in range(1024):
                curr_action = epsilon_greedy_policy(curr_state, simulator.get_actions(curr_state))
                reward, next_state = simulator.get_next_reward_state(curr_state, curr_action)

                # Q-learning update
                old_value = Q.get((curr_state, curr_action), 0.0)
                next_actions = simulator.get_actions(next_state)
                best_next = max(Q.get((next_state, a), 0.0) for a in next_actions)
                Q[(curr_state, curr_action)] = reward + gamma * best_next

                curr_state = next_state
                if reward > 0 or curr_state == simulator.terminal_state:
                    break  # stop rollout after first reward

    # --- Extract optimal policy from Q-table ---
    all_states = set(s for (s, a) in Q.keys())
    for state in all_states:
        actions = simulator.get_actions(state)
        q_values = [Q.get((state, a), 0.0) for a in actions]
        max_q = max(q_values)
        best_actions = [a for a, qv in zip(actions, q_values) if qv == max_q]
        best_action = random.choice(best_actions)
        f.write(f"{state}\t{best_action}\n")



In [32]:
google.colab.files.download("policy-optimal.tsv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Submit "policy-optimal.tsv" in Gradescope.

## Part 6: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

## Part 7: Acknowledgements

If you discussed this assignment with anyone, please acknowledge them here.
If you did this assignment completely on your own, simply write none below.

If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for. If you did not use any other libraries, simply write none below.

If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy. If you did not use any generative AI tools, simply write none below.